In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt

import sys
sys.path.append(os.path.abspath("../../framework"))
sys.path.append(os.path.abspath("../../concrete"))
sys.path.append(os.path.abspath("../../snippet"))

In [ ]:
from ConcAgent import ConcAgent
from ConcEnvironment import ConcEnvironment
from ConcAgentFactory import ConcAgentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from ConcObservation import ConcObservation
from framework import  ObservationSequence 
from framework import TrainId, StoreField
from workUtils import *

# 10. Misc

## 12. load trainlog into a database

In [ ]:
trainLogFolderPath = "./trainLog"
cur = loadTrainLog(trainLogFolderPath)

# Table TrainLog(
#     buildOrderId Text, /* accord to a build order */
#     trainId   Text, /* accord to a trained agent along with a simulation time*/
#     timeSimulation Integer,
#     timestamp timestamp
# )

## 14. a function to load gain and bias from the trains over a training:

In [ ]:
def getGainAndBiasOverTrain(buildOrderIdGiven):

    concStore = ConcStore()
    agentFactory = ConcAgentFactory()

    y = np.random.randn(1, ConcEnvironment.nPv).astype(np.float32)
    anObservationSequence = ObservationSequence()
    anObservationSequence.add(ConcObservation(y))

    cur.execute('''
    Select
        count(*)
        From TrainLog
        Where buildOrderId = ?
        ''', (buildOrderIdGiven,))
    assert cur.fetchone()[0] > 0, "NO DATA WITH THE GIVEN BUILD ID"

    cur.execute('''
    Select 
        trainId
        , timeSimulation
        , timestamp
        From TrainLog
        Where buildOrderId = ?
        Order By timeSimulation
        ''', (buildOrderIdGiven,))


    TimeSimulation = []
    Gain = []
    Bias = []

    for (trainIdStr, timeSimulation, timestamp) in cur.fetchall():

        # storeField contains: agentMement, buildOrder
        storeField = concStore.load(TrainId(trainIdStr))

        # restore agent
        agent = agentFactory.create(storeField.getBuildOrder())
        agent.loadFromMemento(storeField.getAgentMemento())
        agent(anObservationSequence)

        TimeSimulation.append(timeSimulation)
        
        gain = agent.gainP.weights[0].numpy()
        if len(agent.gainP.weights) > 1:
            bias = agent.gainP.weights[1].numpy()
        else:
            bias = np.nan
            
        Gain.append(gain)
        Bias.append(bias)

    TimeSimulation = np.array(TimeSimulation) # (*)
    Gain = np.concatenate(Gain, axis=0) # (*, nMv)
    Bias = np.stack(Bias, axis=0) # (*, nMv)
    weightOnError = storeField.getBuildOrder().weightOnError
    
    return TimeSimulation, Gain, Bias, weightOnError

# 20. Observe the trace of gain and bias over a traing

## 22. Choose a build order

In [ ]:
cur.execute('''
Select
    distinct buildOrderId
    From TrainLog
    Where timestamp >= datetime('2020-05-22 15:00:00')
    Order by timestamp
    ''')

buildOrderIdsToExport = [elm for elm, in cur.fetchall()]

## 24. Load the parameters => SKIPPED

## 26. Plot the trace of Gain and Bias along with the simulation time => SKIPPED

## 28. Export the dataset of gains

Choose a set of iterations over the training

In [ ]:
TimeSimulationsToExport = [255, 511, 1023]

Export the gain with respect to the pair of (training iteration, weightOnError)...

In [ ]:
txt = ["value,epoch,weightOnError",]
for buildOrderIdGiven in buildOrderIdsToExport:
    TimeSimulation, Gain, _, weightOnError = getGainAndBiasOverTrain(buildOrderIdGiven)
    for t, gain in zip(TimeSimulation, Gain[:,0]):
        if t in TimeSimulationsToExport:
            txt.append("{0},{1},weightOnError={2}".format(gain,t,weightOnError))

with open("./tmp/gain.csv", "w") as fp:
    fp.write("\n".join(txt))